In [1]:
import numpy as np 
import pandas as pd
import os 
import sys
import time
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')
print(tf.__version__)

2.3.1


## 一、引入数据集

In [2]:
#引入keras里面的dataset，这是一个电影评分数据集
imdb=keras.datasets.imdb
vocab_size=10000
index_from = 3
(train_data, train_labels), (test_data, test_labels) = imdb.load_data( 
    num_words = vocab_size,index_from = index_from)

In [3]:
train_data.shape

(25000,)

In [4]:
train_labels.shape

(25000,)

## 二、构造完整的句子

In [5]:
#映入词表
word_index = imdb.get_word_index()

In [6]:
len(word_index)

88584

In [7]:
#因为在设定了起始位置从3开始，因为word_index的索引总体都要往后偏移三个单位
word_index = {k:(v+3) for k, v in word_index.items()}

In [8]:
#因为设定了偏移量为3所以在前3个量中可以插入特殊字符
word_index['<PAD>'] = 0  #做padding的时候用来填充的字符
word_index['<START>'] =1  #每个句子开始之前插入一个起始字符
word_index['<UNK>'] = 2  #找不到字符的时候返回UNK
word_index['<END>'] = 3  #在每个句子的末尾加入一个特殊字符

reverse_word_index =dict([(value,key) for key,value in word_index.items()])

def decode_review(text_ids):
    return " ".join(
        [reverse_word_index.get(word_id,"<UNK>") for word_id in text_ids])
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [9]:
max_length = 500

train_data = keras.preprocessing.sequence.pad_sequences(
    train_data,  #list of list
    value = word_index['<PAD>'],
    padding = 'post',  #post,pre
    maxlen = max_length
)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data,  #list of list
    value = word_index['<PAD>'],
    padding = 'post',  #post,pre
    maxlen = max_length
)

In [10]:
print(train_data[0])

[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4 

In [12]:
embedding_dim = 16
batch_size = 128

single_rnn_model = keras.models.Sequential([   
    keras.layers.Embedding(vocab_size,
                           embedding_dim,
                           input_length=max_length),
    keras.layers.SimpleRNN(units = 64, 
                           return_sequences=False),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid'),
])

In [13]:
single_rnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           160000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                5184      
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 169,409
Trainable params: 169,409
Non-trainable params: 0
_________________________________________________________________


In [14]:
single_rnn_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [15]:
history = single_rnn_model.fit(
    train_data,
    train_labels,
    epochs=30,
    batch_size=batch_size,
    validation_split=0.2)

Epoch 1/30
157/157 [==============================] - 32s 205ms/step - loss: 0.6944 - accuracy: 0.5030 - val_loss: 0.6933 - val_accuracy: 0.4954
Epoch 2/30
157/157 [==============================] - 33s 209ms/step - loss: 0.6949 - accuracy: 0.5048 - val_loss: 0.6931 - val_accuracy: 0.4946
Epoch 3/30
157/157 [==============================] - 32s 206ms/step - loss: 0.6931 - accuracy: 0.5053 - val_loss: 0.6937 - val_accuracy: 0.4944
Epoch 4/30
157/157 [==============================] - 33s 208ms/step - loss: 0.6862 - accuracy: 0.5289 - val_loss: 0.6960 - val_accuracy: 0.5070
Epoch 5/30
157/157 [==============================] - 33s 209ms/step - loss: 0.6682 - accuracy: 0.5386 - val_loss: 0.6984 - val_accuracy: 0.5028
Epoch 6/30
157/157 [==============================] - 34s 216ms/step - loss: 0.6516 - accuracy: 0.5339 - val_loss: 0.7039 - val_accuracy: 0.5018
Epoch 7/30
157/157 [==============================] - 33s 213ms/step - loss: 0.6455 - accuracy: 0.5469 - val_loss: 0.7173 - val_ac

In [ ]:
single_rnn_model.evaluate(test_data,test_labels,
                batch_size=batch_size)